In [ ]:
import torch
import torch.nn.functional as F
import numpy as np

batch_size = 2
src_len = torch.Tensor([2, 4]).to(torch.int32)
tgt_len = torch.Tensor([4, 3]).to(torch.int32)

########################## Step1: 单词索引构成源句子和目标句子，构建patch，并且做了padding，默认值为0  ##########################

########################## Step2：构造word embedding  ##########################

########################## Step3：构造positional encoding  ##########################

########################## Step4：构造encoder的self-attention mask ##########################
# mask的shape：[batch_size, max_src_len, max_src_len]，值为1或-inf

########################## Step5：构造intra-attention的mask ##########################
# Q @ K^T shape: [batch_size, tgt_seq_len, src_seq_len]
vaild_encoder_pos = torch.unsqueeze(torch.cat([torch.unsqueeze(F.pad(torch.ones(L), (0, max(src_len)-L)), 0) for L in src_len], dim=0), 2) # 为什么最后一个参数为2？因为矩阵运算是不会让batch_size这个维度参与的，只会让seq_len这个维度参与，因此需要对非0维剩下的维度进行扩维
vaild_decoder_pos = torch.unsqueeze(torch.cat([torch.unsqueeze(F.pad(torch.ones(L), (0, max(tgt_len) - L)), 0) for L in tgt_len], dim=0), 2)


vaild_cross_pos_matrix = torch.bmm(vaild_decoder_pos, vaild_encoder_pos.transpose(1, 2)) # Q @ K^T，为什么要有.transpose(1, 2)？vaild_encoder_pos.shape和vaild_decoder_pos.shape都是torch.Size([2, 4, 1])，bmm()矩阵乘法计算就要对矩阵进行转置
invaild_cross_pos_matrix = 1 - vaild_cross_pos_matrix

mask_cross_attention = invaild_cross_pos_matrix.to(torch.bool)
# # print(vaild_encoder_pos) # '1'表示真实的token，而'0'表示是由padding来的
# print("The shape of vaild_encoder_loss is: ", vaild_encoder_pos.shape)

# print(vaild_decoder_pos)
# print("The shape of vaild_decoder_pos is : ", vaild_decoder_pos.shape)

# print(vaild_cross_pos_matrix)
# print("The shape of vaild_cross_pos is : ", vaild_cross_pos_matrix.shape)

# print(invaild_cross_pos_matrix)

# print(mask_cross_attention)
# print("The shape of mask_cross_attention is : ", mask_cross_attention.shape)

########################## Step 6：构造decoder self-attention的mask ##########################
tri_matrix = [torch.tril(torch.ones((L, L))) for L in tgt_len] # 还需要进行padding操作，因为长度不一样

# print(tri_matrix)
# for l in tri_matrix:
#     print(l.shape)

# 对长度不一的tri_matrix进行padding操作
vaild_decoder_tri_matrix = torch.cat([torch.unsqueeze(F.pad(torch.tril(torch.ones((L, L))), (0, max(tgt_len)-L, 0, max(tgt_len)-L)), 0) for L in tgt_len], dim=0)
vaild_decoder_tri_matrix_wrong = torch.cat([F.pad(torch.tril(torch.ones((L, L))), (0, max(tgt_len)-L)) for L in tgt_len], dim=0) # 对比上面这行正确的代码，一定要想清楚为什么这样不对

# print(vaild_decoder_tri_matrix)
# print("The shape of vaild_decoder_tri_matrix is: {}".format(vaild_decoder_tri_matrix.shape))
# print("-"*100)
# print(vaild_decoder_tri_matrix_wrong)
# print("The shape of vaild_decoder_tri_matrix_wrong is: ", vaild_decoder_tri_matrix_wrong.shape)

invaild_decoder_tri_matrix = (1 - vaild_decoder_tri_matrix).to(torch.bool) # 表示是否需要对该位置进行mask以符合因果律，True表示需要mask

# print(invaild_decoder_tri_matrix)
# print("The shape of invaild_decoder_tri_matrix is : ", invaild_decoder_tri_matrix.shape)

score = torch.randn(batch_size, max(tgt_len), max(tgt_len))
# 测试masked的效果
masked_score = score.masked_fill(invaild_decoder_tri_matrix, -1e9) # masked_fill 函数会检查 mask 中的每个元素。如果 mask 中的元素为 True，则在输出张量中用 value 替换输入张量中对应位置的元素；如果 mask 中的元素为 False，则保留输入张量中对应位置的元素
# print("-"*100)
# print(masked_score)
# print("The shape of masked_score is : ", masked_score.shape)

prob = F.softmax(masked_score, -1)
# print("-"*100)
# print("tgt_len = ", tgt_len)
# print("prob = ", prob)


########################## Step 7: 构建scaled self-attention ##########################
def scaled_dot_product_attention(Q, K, V, attn_mask):
    # shape of Q,K,V : (batch_size*num_head, seq_len, model_dim/num_head)
    score = torch.bmm(Q, K.transpose(-2, -1)) / torch.sqrt(model_dim)
    masked_score = score.masked_fill(attn_mask, -1e9)
    prob = F.softmax(masked_score, -1)
    context = torch.bmm(prob, V)
    return context


tensor([[[False,  True,  True,  True],
         [False, False,  True,  True],
         [False, False, False,  True],
         [False, False, False, False]],

        [[False,  True,  True,  True],
         [False, False,  True,  True],
         [False, False, False,  True],
         [ True,  True,  True,  True]]])
The shape of invaild_decoder_tri_matrix is :  torch.Size([2, 4, 4])
----------------------------------------------------------------------------------------------------
tensor([[[-1.3444e+00, -1.0000e+09, -1.0000e+09, -1.0000e+09],
         [-1.8432e-01,  9.3469e-01, -1.0000e+09, -1.0000e+09],
         [-2.1944e+00, -7.9379e-01,  2.8880e+00, -1.0000e+09],
         [ 1.1420e-01,  8.0203e-01,  1.7032e+00, -6.3980e-01]],

        [[-6.9993e-01, -1.0000e+09, -1.0000e+09, -1.0000e+09],
         [ 9.5286e-02, -7.1054e-01, -1.0000e+09, -1.0000e+09],
         [ 1.6955e+00, -1.1115e+00,  1.4929e-01, -1.0000e+09],
         [-1.0000e+09, -1.0000e+09, -1.0000e+09, -1.0000e+09]]])
The sha